In [ ]:
import $ivy.`org.typelevel::cats-core:2.1.0`

// These are all the imports you need for everything here
import cats.implicits._
import cats.{Id, Monad}

import scala.concurrent.Future

# Monads in Cats
## Monad Type Class
The monad type class is `cats.Monad`. It extends two other type classes: `FlatMap`, which provides the `flatMap` method, and `Applicative`, which provides, amongst other things, the `pure` function. `Applicative` also extends `Functor`, which gives every `Monad` a `map` method as we just saw in the exercise.

Here are some examples:

In [ ]:
val opt1 = Monad[Option].pure(3)
val opt2 = Monad[Option].flatMap(opt1)(a => Some(a + 2))
val opt3 = Monad[Option].map(opt2)(a => 100 * a)

val list1 = Monad[List].pure(3)
val list2 = Monad[List].flatMap(List(1, 2, 3))(a => List(a, a * 10))
val list3 = Monad[List].map(list2)(_ + 123)

`Monad` also provides many other userful methods, including all the methods from `Applicative` and `Functor`.

## Default Instances
Cats also provides instances for all the monads in the standard library (`Option`, `List`, `Vector`, etc).

Cats also provides a `Monad` for `Future`. Unlike the methods on the `Future` class itself, the `pure` and `flatMap` methods on the monad can’t accept implicit `ExecutionContext` parameters (because the parameters aren’t part of the
definitions in the `Monad` trait). To work around this, Cats requires us to have an `ExecutionContext` in scope when we summon a `Monad` for `Future`:

In [ ]:
val fm = Monad[Future]

In [ ]:
import scala.concurrent.ExecutionContext.Implicits.global

val fm = Monad[Future]
// The Monad instance uses the captured ExecutionContext for subsequent calls to pure and flatMap

## Monad Syntax
We can use `pure` to construct instances of a monad. We’ll often need to specify the type parameter to disambiguate the particular instance we want.

In [ ]:
val opt  = 1.pure[Option]
val list = 1.pure[List]

It’s difficult to demonstrate the `flatMap` and `map` methods directly on Scala monads like `Option` and `List`, because they define their own explicit versions of those methods. Instead we’ll write a generic function that performs a calculation on parameters that come wrapped in a monad of the user’s choice:

In [ ]:
def sumSquare[F[_]: Monad](a: F[Int], b: F[Int]): F[Int] =
  for {
    x <- a
    y <- b
  } yield x*x + y*y

sumSquare(3.some, 4.some)

sumSquare(List(1, 2, 3), List(4, 5))

## The Identity Monad
The `sumSquare` method is pretty neat. What if we want to use it with plain old values?

In [ ]:
sumSquare(3, 4)

Quite a lovely error message isn't it? It would be awesome if we could use it with monadic and non-monadic code. Fortunately, Cats provides the `Id` type to bridge the gap:

In [ ]:
sumSquare[Id](3, 4)

Interesting. Lets take a look at the definition of `Id`:

In [ ]:
// Calling it MyId to not overwrite cats.Id
type MyId[A] = A

It's just a type alias to turn a type into a single-parameter type constructor. You can cast anything to the corresponding `Id`. Cats provides instances for various type classes for `Id`, including `Functor` and `Monad`:

In [ ]:
val stringId = "Dave" : Id[String]
val intId    = 123 : Id[Int]
val listId   = List(1, 2, 3) : Id[List[Int]]

val a = Monad[Id].pure(3)
val b = Monad[Id].flatMap(a)(_ + 1)

val c =
  for {
    x <- a
    y <- b
  } yield x + y

### Exercise
Lets try to implement `Monad` for `Id`:

In [ ]:
// Ignore StackSafeMonad for now
implicit val monadForId: Monad[Id] = new Monad[Id] with cats.StackSafeMonad[Id] {
  override def pure[A](value: A): Id[A] =
    ???
  override def map[A, B](initial: Id[A])(func: A => B): Id[B] =
    ???
  override def flatMap[A, B](initial: Id[A])(func: A => Id[B]): Id[B] =
    ???
}

In [ ]:
// Ignore StackSafeMonad for now
implicit val monadForId: Monad[Id] = new Monad[Id] with cats.StackSafeMonad[Id] {
  override def pure[A](value: A): Id[A] =
    value
  override def map[A, B](initial: Id[A])(func: A => B): Id[B] =
    func(initial)
  override def flatMap[A, B](initial: Id[A])(func: A => Id[B]): Id[B] =
    func(initial)
}

This ties in with our understanding of functors and monads as sequencing type classes. Each type class allows us to sequence operations ignoring some kind of complication. In the case of `Id` there is no complication, making `map` and
`flatMap` the same thing.

## Either
In 2.11 and earlier, `Either` wasn't really a monad since it didn't have `map` and `flatMap` methods. In 2.12 it became *right biased*. Now `Either` makes the decision that the right side represents the success case and supports `map` and `flatMap` directly. Cats back-ports this behavior to 2.11 by implementing `Monad` for `Either`, allowing you to call `map` and `flatMap` on it as you would in 2.12.

### Either Syntax
You can create instances of `Either` with `asRight` and `asLeft`:

In [ ]:
val a = 3.asRight[String]
val b = 4.asRight[String]

val result = for {
  x <- a
  y <- b
} yield x*x + y*y

These "smart constructors" have advantages over `Left.apply` and `Right.apply`, similar to the advantages of `value.some` over `Some(value)`. They return type `Either` instead of `Left` and `Right` which helps avoid inference bugs cause by over-narrowing.

Cats adds useful extension methods to `Either` and the `Either` companion object:

In [ ]:
val a = Either.catchOnly[NumberFormatException]("foo".toInt)
val b = Either.catchNonFatal(sys.error("Badness"))

// We can also create Either form other types
val fromTry = Either.fromTry(scala.util.Try("foo".toInt))
val fromOpt = Either.fromOption[String, Int](None, "Badness")

// getOrElse and orElse are added
val getOrElse = "Error".asLeft[Int].getOrElse(0)
val orElse    = "Error".asLeft[Int].orElse(2.asRight[String])

// ensure allows us to check a predicate and return Left if it isn't satisfied
val ensure = -1.asRight[String].ensure("Must be non-negative!")(_ > 0)

// We get recover and recoverWith, similar to Future
val recover = "error".asLeft[Int].recover {
  case str => -1
}

val recoverWith = "error".asLeft[Int].recoverWith {
  case str => Right(-1)
}

// We get leftMap and bimap to compliment map
val leftMap = "foo".asLeft[Int].leftMap(_.reverse)
val bimap1  = 6.asRight[String].bimap(_.reverse, _ * 7)
val bimap2  = "bar".asLeft[Int].bimap(_.reverse, _ * 7)

// Swap lets up switch the left and right
val swap = 123.asRight[String].swap

// Finally, there is a bunch of conversion methods
val toOpt  = a.toOption
val toList = a.toList
val toTry  = a.toTry
val toVal  = a.toValidated

## Error Handling and MonadError
The book says this chapter is optional so I'll give it a quick summary. You can abstract over error handling behavior similar to that seen in `Either`, `Try`, and `Future` with the type class `MonadError`. Here is a simplified definition if you're curious:

In [ ]:
// Calling it MyMonadError to not overwrite cats.MonadError
trait MyMonadError[F[_], E] extends Monad[F] {
  def raiseError[A](e: E): F[A]
  def handleErrorWith[A](fa: F[A])(f: E => F[A]): F[A]
  def ensure[A](fa: F[A])(error: => E)(predicate: A => Boolean): F[A] =
    flatMap(fa)(a => if (predicate(a)) pure(a) else raiseError(error))
  // And plenty more utility methods...
}